<a href="https://colab.research.google.com/github/muyeblog/implementAlgorithmFromScratch/blob/main/LogisticRegression_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries for logistic Regression

参看：https://www.analyticsvidhya.com/blog/2022/02/implementing-logistic-regression-from-scratch-using-python/

In [ ]:
import numpy as np
from numpy import log,dot,e,shape
import matplotlib.pyplot as plt

import dataset

In [ ]:
from sklearn.datasets import make_classification
X,y = make_classification(n_features=4,n_classes=2)
from sklearn.model_selection import train_test_split
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.1)

## Standardization
Standardization is the process of scaling data around the mean with a unit standard deviation(使用单位标准差围绕均值缩放数据).That means we are effectively making the mean of the attribute zero with the resulting distribution having a standard deviation equal to zero(这意味着我们有效的使属性的平均值为 0，标准差分布为 0).Some algorithms are vulnerable to features with different scales.Especially if we are using gradient descent for optimization, then the model will have a hard time giving accurate results; for example,if a dataset has two features, age and salary, then the salary feature with its higher range will most likely dominate(主导) the outcome. So,it is a good practice to standardize data before feeding it to the algorithm. Highly recommended to go through this article to understand standardization clearly. Mathematically it it given as 

(x-μ)/σ

This is often necessary when attributes are from different scales.

In [ ]:
def standardize(X_tr):
  for i in range(shape(X_tr)[1]):
    X_tr[:,i] = (X_tr[:,i] - np.mean(X_tr[:,i]))/np.std(X_tr[:,i])

## Initializing Parameters

The data sets are always multidimensional. We will need to use matrics for any kind of calculation. So,for input,we have two matrices to deal with.The first one is for feature vectors, and the second is for parameters or weights. Our first matrix is of the m x n dimension, where m is the number of observations while n is the dimension of observations. And the second on is of n x 1 dimension. Here, we will add a bias column of ones to our feature vectors matrix and a corresponding parameter term to weight vector. Bias is import to make the model more flexible.

In [ ]:
def initialize(self,X):
  weights = np.zeros((shape(X)[1]+1,1))
  X = np.c_[np.ones((shape(X)[0],1)),X]
  return weights,X

In [ ]:
# demo:
demo = np.array([[1,2,3],[4,5,6]])
print(shape(demo))
print(np.ones((shape(demo)[0],1))) # 创建了与样本数量一致的bias为 1 的数组
print(np.c_[np.ones((shape(demo)[0],1)),demo]) # 将 bias数组与样本数组拼接

(2, 3)
[[1.]
 [1.]]
[[1. 1. 2. 3.]
 [1. 4. 5. 6.]]


In [ ]:
np.c_[demo,np.ones((shape(demo)[0],1))]
# np.c_是按行连接两个矩阵，就是把两矩阵左右相拼接，要求行数相等。

array([[1., 2., 3., 1.],
       [4., 5., 6., 1.]])

Note: In the code above, although we have initialized the weight vector to be a vector of zeros, you could opt for any other values as well.



## Sigmoid Function
In a linear regression model,the hypothesis function is a linear combination of parameters given as y=ax+b for a simple single parameter data. This allows us to predict continuous values effectively,but in logistic regression, the response variables are binomial, either 'yes' or 'no'. So, it makes less sense to use the linear function to predict anything except the values between 0 and 1. And the most effective function to limit results of a linear equation to  [0,1] is the sigmoid or logistic function.

![](https://editor.analyticsvidhya.com/uploads/22366480px-Logistic-curve.svg.png)

As you can see , the sigmoid function intersects the y-axis at 0.5. In most cases, we  use this point as a threshold for classification. Any value above it will be classified as 1, while any value below is 0. This is not a rule of thumb(这不是经验法则). We can also use different values instead of 0.5 , depending on the requirements. The sigmoid function:

$y = 1 / (1 + e^{-x})$

We plug the linear equation in  place of x.

$y = 1/(1 + e^{-(ax+b)})$

In [ ]:
def sigmoid(self,z):
  sig = 1/(1 + e**(-z))
  return sig

In the above expression, z is the dot product of m x n matrix containing observation and n x 1 matrix of weights.

## Cost Function
Cost function or loss function is that function that describes how much the calculated value deviates from the actual value. Linear regression employs the least squared error as the cost function. But the least squared error function for logistic regression is non-convex(非凸的). While performing gradient descent chances that we get stuck in a local minimum is more. So instead, we use log loss as the cost function.

![](https://editor.analyticsvidhya.com/uploads/67304NN_C4.png)

The formula gives the cost function for the logistic regression.

$J(\theta) = -\frac{1}{m}\sum_{i=1}^{m}[y^{(i)}\log(h_{\theta}(x^{(i)})) +(1-y^{(i)})\log(1-h_{\theta}(x^{(i)})]$

where $h(x)$ is the sigmoid function we used earlier.

In [ ]:
def cost(self,theta):
  z = dot(X,theta)
  cost0 = y.T.dot(log(self.sigmoid(z)))
  cost1  = (1-y).T.dot(log(1-self.sigmoid(z)))
  cost = -((cost1 + cost0))/len(y)
  return cost

## Gradient Descent
The next step is gradient descent. Gradient descent is an optimization algorithm that is responsible for the learning of best-fitting parameters. So what are the gradients? The gradients are the vector of the 1st order derivative of the cost function.These are the direction of the steepest ascent or maximum of a function. For gradient descent,we move in the opposite direction of the gradients. We will bee updating the weights in every iteration until the convergence. The pseudocode for Gradient descent looks something like this.

![](图裂)

Here,the alpha is the step size responsible for how quick it converages to the global minimum. If the step size is too small, it will converage slowly, but if it is too large, it may overshoot the minimum while descending.

By differentiating the cost function, we get the gradient descent expression

$\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{2m}\sum_{i=1}^m (h(x^{(i)})-y)x_j^{(i)}$

or

$X^T(h_{\theta}(x) - y)$

This is vectorised form of the gradient expression, which we will be using in our code.

In [ ]:
def fit(self,X,y,alpha=0.001,iter=100):
  params,X = self.initialize(X)
  cost_list = np.zeros(iter,)
  for i in range(iter):
    params = params - alpha * dot(X.T,self.sigmoid(dot(X,params)) - np.reshape(y,(len(y),1)))
    cost_list[i] = cost(params)
  self.params = params
  return cost_list


## Prediction 
Everything that we have done far is for this step. We trained the model on a training dataset, and we will use the learned parameters to predict the unseen data.

In [ ]:
def predict(self,X):
  z = dot(self,initialize(X)[1],self.weights)
  lis = []
  for i in self.sigmoid(z):
    if i > 0.5:
      lis.append(1)
    else:
      lis.append(0)
  return lis

## F1-Score
Now that we are done with the prediction, we will move on to the F1-score section, where we will measure how good our model predicts for unseen data. The F1-score is a robust metric for evaluating the performances of classification modelss, and mathematically F1-score is the harmonic mean of precision and recall(从数学上来讲，F1-score是准确率和召回率的调和平均值).

Recall, Precision,F1 Score - Simple Metric Explanation Machine Learning

precision = Precision is the number of true positives over the sum of true positives and false positives.
precision = TP/(TP + FP)

recall =  Recall is the number of true positives over  the sum of true postive positives and false negatives.
recall = TP/(TP + FN)

In [ ]:
def f1_score(y,y_hat):
  tp,tn,fp,fn=0,0,0,0
  for i in range(len(y)):
    if y[i] == 1 and y_hat[i]==1:
      tp += 1
    elif y[i]==1  and y_hat[i]==0:
      fn += 1
    elif y[i]==0 and  y_hat[i]==1:
      fp += 1
    elif y[i]==0 and y_hat[i]==0:
      tn += 1
  precision = tp /(tp + fp)
  recall = tp/(tp + fn)
  f1_score = 2*precision*recall/(precision + recall)
  return f1_score

## Putting Everything Together: Logistic Regression

Now that we are done with  every part, we will put everything together in a single class.

In [ ]:
import numpy as np
from numpy import log,dot,exp,shape
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

X,y = make_classification(n_features=4)
from sklearn.model_selection import train_test_split
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.1)

In [ ]:
import numpy as np
from numpy import log,dot,exp,shape
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

X,y = make_classification(n_features=4)
from sklearn.model_selection import train_test_split
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.1)


def standardize(X_tr):
  for i in range(shape(X_tr)[1]):
    X_tr[:,i] = (X_tr[:,i] - np.mean(X_tr[:,i]))/np.std(X_tr[:,i])

def F1_score(y,y_hat):
  tp,tn,fp,fn = 0,0,0,0
  for i in range(len(y)):
    if y[i] == 1 and y_hat[i] == 1:
      tp += 1
    elif y[i] == 1 and y_hat[i] == 0:
      fn += 1
    elif y[i] == 0 and y_hat[i] == 1:
      fp += 1
    elif y[i] == 0 and y_hat[i] == 0:
      tn += 1
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_score = 2*precision*recall / (precision + recall)
  return  f1_score

class LogisticRegression:
  def sigmoid(self,z):
    sig = 1 / (1 + exp(-z))
    return sig
  
  def initialize(self,X):
    weights = np.zeros((shape(X)[1]+1,1))
    X = np.c_[np.ones((shape(X)[0],1)),X] # 添加并拼接上bias系数列
    return  weights,X
  
  def fit(self,X,y,alpha=0.01,iter=400):
    weights,X = self.initialize(X)
    def cost(theta):
      # dot 为矩阵乘法
      z = dot(X,theta) # 一个样本一个z，z的 shape 为 (sample_size,1)
      # y shape (sample_size,1) y.T shape (1,sample_size)
      cost0 = y.T.dot(log(self.sigmoid(z))) 
      cost1 = (1-y).T.dot(log(1-self.sigmoid(z)))
      # print('cost0',cost0)
      cost = -(cost0 + cost1)/len(y)
      # print('cost',cost)
      # print(cost)
      return cost
    cost_list = np.zeros(iter,)
    for i in range(iter):
      weights = weights - alpha*dot(X.T,self.sigmoid(dot(X,weights)) - np.reshape(y,(len(y),1)))
      cost_list[i] = cost(weights)
    self.weights = weights
    return cost_list
  
  def predict(self,X):
    z = dot(self.initialize(X)[1],self.weights)
    lis = []
    for i in self.sigmoid(z):
      if i > 0.5:
        lis.append(1)
      else:
        lis.append(0)
    return lis

In [ ]:
X_tr.shape

(90, 4)

In [ ]:
X_tr[:3,]

array([[-0.44733753,  1.46761239, -0.65926372, -1.06834666],
       [ 2.02239535, -0.04916234, -1.0227897 ,  1.08891915],
       [-0.31281433,  0.08904525,  0.10869519, -0.21469059]])

In [ ]:
standardize(X_tr)
standardize(X_te)
obj1 = LogisticRegression()
model = obj1.fit(X_tr,y_tr)

In [ ]:
y_pred = obj1.predict(X_te)
y_train = obj1.predict(X_tr)

In [ ]:
## Let's see the f1-score for training and testing
f1_score_tr = F1_score(y_tr,y_train)
f1_score_te = F1_score(y_te,y_pred)
print(f1_score_tr)
print(f1_score_te)

0.9555555555555556
1.0


Now,let's see how our logistic regression fares in comparison to sklearn's logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
model = LogisticRegression().fit(X_tr,y_tr)
y_pred = model.predict(X_te)
print(f1_score(y_te,y_pred))

1.0


In [ ]:
def sigmoid(z):
  sig = 1/(1 + exp(-z))
  return sig

In [ ]:
y_tr.T.dot(log(sigmoid(X_tr)))

array([-17.84327904, -26.90887447, -54.09310159, -37.03621028])

In [ ]:
np.dot(np.array([[1,2,3]]),np.array([[1,2,3]]).T).shape


(1, 1)

In [ ]:
np.array([[1,2,3]]).shape

(1, 3)

In [ ]:
np.array([[1,2,3]]).T.shape

(3, 1)

In [ ]:
np.array([[1,2,3]]).dot(np.array([[1,2,3]]).T)


array([[14]])

In [ ]:
np.dot(np.array([[1,2,3]]),np.array([[1,2,3]]).T)


array([[14]])

In [ ]:
log(np.array([[1,2,3]]))

array([[0.        , 0.69314718, 1.09861229]])

In [ ]:
X_tr[:5,]

array([[-0.20449986,  0.96212659, -0.60551239, -0.79812379],
       [ 1.18580724,  0.06441484, -0.88098499,  0.74745564],
       [-0.12877161,  0.14621379, -0.02356874, -0.18651939],
       [ 0.09042609, -1.94393966,  1.44955939,  1.39707681],
       [-0.57298926,  0.16115541,  0.27604995, -0.49339226]])

dot 计算矩阵乘法时：
设A是$n\times m$的矩阵，B是$m\times p$的矩阵，则它们的矩阵积AB是$n\times p$的矩阵.

dot 计算向量内积时：
向量是一维矩阵，两个向量进行内积运算时，需要保证两个向量包含的元素个数是相同的

In [ ]:
x = np.array([1,2,3,4,5,6,7])
print(x.shape)  # (7,) 为一维矩阵，即向量的维度
print(x.T.shape)  # 转置前后维度不变 (7,) 为一维矩阵，即向量的维度
y = np.array([2,3,4,5,6,7,8])
result = np.dot(x, y) 
print(result)

(7,)
(7,)
168
